In [1]:
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

Importando a tabela de produção dos items da cesta básica (1974 - 2020)

In [2]:
producao = pd.read_excel('C:\dcc-fellowship-ciencia-de-dados\pam.xlsx')
producao.head()

,Ano,Item,Produçao
0,1974,Arroz,6764038
1,1975,Arroz,7781538
2,1976,Arroz,9757079
3,1977,Arroz,8993696
4,1978,Arroz,7296142


In [3]:
for item in producao.Item.unique():
    print(item)

Arroz 
Café (em grão)
Cana-de-açúcar
Feijão (em grão)
Mandioca
Milho (em grão)
Soja (em grão)
Tomate
Trigo (em grão)


Criando um dataframe para cada item da cesta

In [4]:
df_arroz=producao[producao['Item']=='Arroz '].reset_index(drop = True)
df_cafe=producao[producao['Item']=='Café (em grão)'].reset_index(drop = True)
df_cana=producao[producao['Item']=='Cana-de-açúcar'].reset_index(drop = True)
df_feijao=producao[producao['Item']=='Feijão (em grão)'].reset_index(drop = True)
df_mandioca=producao[producao['Item']=='Mandioca'].reset_index(drop = True)
df_milho=producao[producao['Item']=='Milho (em grão)'].reset_index(drop = True)
df_soja=producao[producao['Item']=='Soja (em grão)'].reset_index(drop = True)
df_tomate=producao[producao['Item']=='Tomate'].reset_index(drop = True)
df_trigo=producao[producao['Item']=='Trigo (em grão)'].reset_index(drop = True)

Cálculo da média móvel para cada item da cesta

* Arroz

In [5]:
## Média móvel
df_arroz['Média móvel'] = df_arroz['Produçao'].rolling(window =5).mean()

In [6]:
fig =px.line(df_arroz, x ='Ano', y = ['Produçao','Média móvel'], title = 'Produção de ARROZ - Brasil - (1974-2020)', labels= {'value': 'Produção (ton)'})
fig.update_layout(template = 'plotly_white')

Regressão linear simples utilizando a coluna Média móvel como resposta

In [7]:
# Declarar as variáveis 
#Arroz
Y_arroz = df_arroz.loc[4:]['Média móvel'].values  
X_arroz = df_arroz.loc[4:][['Ano']].values
X_arroz = sm.add_constant(X_arroz)

In [8]:
#Arroz
modelo_arroz_mm = sm.OLS(Y_arroz,X_arroz)
resultados_arroz_mm = modelo_arroz_mm.fit()
#print(resultados_arroz_mm.summary())

In [9]:
print("Coeficientes: ", resultados_arroz_mm.params)
print("R2: ", resultados_arroz_mm.rsquared)

Coeficientes:  [-1.80831250e+08  9.56639287e+04]
R2:  0.8246475580115764


Utilizando os coeficientes gerados nas regressões para incluir no dataframe a produção de ARROZ prevista de 2020 ate 2030

In [10]:
df_arroz_media_movel = pd.read_excel("C:\dcc-fellowship-ciencia-de-dados\df_arroz_media_move.xlsx")
#df_arroz_media_movel.drop('Unnamed: 3', axis =1, inplace = True)
df_arroz_media_movel =df_arroz_media_movel[df_arroz_media_movel.Ano >2017]
df_arroz_media_movel

,Unnamed: 0,Ano,Item,Produçao,Média móvel
44,44,2018,Arroz,11808412,11874434.0
45,45,2019,Arroz,10368639,11513041.4
46,46,2020,Arroz,11091011,11271003.4


Filtrando os registros onde a coluna ANO é maior que 2018, já que estamos levando em consideração os padrões alimentares de consumo indicados pela POF (2018)

In [11]:
df_arroz = df_arroz[df_arroz['Ano']>2017]
df_arroz

,Ano,Item,Produçao,Média móvel
44,2018,Arroz,11808412,11874434.0
45,2019,Arroz,10368639,11513041.4
46,2020,Arroz,11091011,11271003.4


Demanda populacional por ARROZ em toneladas (2018 -2030)

In [12]:
projecao_demanda_brasil = pd.read_excel("C:\dcc-fellowship-ciencia-de-dados\projecao_demanda_brasil.xlsx")
projecao_demanda_brasil.head()

,Unnamed: 0,Ano,Região,Populacao (mi),Items,Consumo médio per capita (kg),Demanda pop (ton)
0,0,2018,Brasil,208494900,Arroz polido,17.065000,3.557965e+06
1,1,2018,Brasil,208494900,Feijão,4.983333,1.039000e+06
2,2,2018,Brasil,208494900,Farinha de mandioca,2.030000,4.232446e+05
3,3,2018,Brasil,208494900,Farinha de trigo,2.311667,4.819707e+05
4,4,2018,Brasil,208494900,Fubá de milho,1.120000,2.335143e+05


In [13]:
df_demanda_arroz = projecao_demanda_brasil[projecao_demanda_brasil['Items'] == ' Arroz polido'][['Ano', 'Items', 'Região', 'Demanda pop (ton)']].reset_index(drop = True)
df_demanda_arroz.head()

,Ano,Items,Região,Demanda pop (ton)
0,2018,Arroz polido,Brasil,3.557965e+06
1,2019,Arroz polido,Brasil,3.586161e+06
2,2020,Arroz polido,Brasil,3.613611e+06
3,2021,Arroz polido,Brasil,3.640266e+06
4,2022,Arroz polido,Brasil,3.666049e+06


In [14]:
df_arroz_media_movel  = pd.merge(df_arroz_media_movel, df_demanda_arroz, how = 'inner', on = 'Ano')

In [15]:
df_arroz

,Ano,Item,Produçao,Média móvel
44,2018,Arroz,11808412,11874434.0
45,2019,Arroz,10368639,11513041.4
46,2020,Arroz,11091011,11271003.4


Juntando os dataframe df_arroz e df_demanda_arroz

In [16]:
df_arroz = pd.merge(df_arroz, df_demanda_arroz, how = 'inner', on = 'Ano')

In [17]:
df_arroz

,Ano,Item,Produçao,Média móvel,Items,Região,Demanda pop (ton)
0,2018,Arroz,11808412,11874434.0,Arroz polido,Brasil,3.557965e+06
1,2019,Arroz,10368639,11513041.4,Arroz polido,Brasil,3.586161e+06
2,2020,Arroz,11091011,11271003.4,Arroz polido,Brasil,3.613611e+06


In [18]:
df_arroz.to_excel('df_arroz_demanda_prod.xlsx')

In [19]:
fig =px.scatter(df_arroz_media_movel, x ='Ano', y = ['Média móvel', 'Demanda pop (ton)'], title = 'Predição da produção e demanda de ARROZ - Brasil - (2018-2030)', labels= {'value': 'Produção (ton)'})
fig.update_layout(template = 'plotly_white')

In [20]:
df_arroz_media_movel

,Unnamed: 0,Ano,Item,Produçao,Média móvel,Items,Região,Demanda pop (ton)
0,44,2018,Arroz,11808412,11874434.0,Arroz polido,Brasil,3.557965e+06
1,45,2019,Arroz,10368639,11513041.4,Arroz polido,Brasil,3.586161e+06
2,46,2020,Arroz,11091011,11271003.4,Arroz polido,Brasil,3.613611e+06


* Feijão

In [21]:
## Média móvel
df_feijao['Média móvel'] = df_feijao['Produçao'].rolling(window =5).mean()

In [22]:
fig =px.line(df_feijao, x ='Ano', y=['Produçao','Média móvel'], title = 'Produção de FEIJÃO - Brasil - (1974-2020)', labels= {'value': 'Produção (ton)'})
fig.update_layout(template = 'plotly_white')

Regressão linear simples utilizando a coluna Produçao como resposta

In [23]:
#Defininição das variáveis
Y_feijao = df_feijao['Produçao'].values
X_feijao = df_feijao[['Ano']].values
X_feijao = sm.add_constant(X_feijao)

In [24]:
#Modelo
modelo_feijao = sm.OLS(Y_feijao,X_feijao)
resultados_feijao = modelo_feijao.fit()
#print(resultados_cafe.summary())

In [25]:
print("Coeficientes: ", resultados_feijao.params)
print("R2: ", resultados_feijao.rsquared)

Coeficientes:  [-4.65350278e+07  2.46646230e+04]
R2:  0.5111822062659961


Regressão linear simples utilizando a coluna Média móvel como variável resposta

In [26]:
# Declarar as variáveis 
#Arroz
Y_feijao= df_feijao.loc[4:]['Média móvel'].values  
X_feijao = df_feijao.loc[4:][['Ano']].values
X_feijao = sm.add_constant(X_feijao)

In [27]:
#Modelo
modelo_feijao_mm = sm.OLS(Y_feijao,X_feijao)
resultados_feijao_mm = modelo_feijao_mm.fit()
#print(resultados_feijao_mm.summary())

In [28]:
print("Coeficientes: ", resultados_feijao_mm.params)
print("R2: ", resultados_feijao_mm.rsquared)

Coeficientes:  [-5.05970273e+07  2.66786866e+04]
R2:  0.7875152128257473


Utilizando os coeficientes gerados nas regressões para incluir no dataframe a produção de FEIJÂO previsto de 2020 ate 2030

In [29]:
ano = 2020
for i in range(10):
    ano += 1 
    df_feijao.loc[len(df_feijao)] = [ano, 'Feijão (em grão)', 24664*(ano) -46535027.8, 26678*(ano)-50597027]

Filtrando os registros onde a coluna ANO é maior que 2018, já que estamos levando em consideração os padrões alimentares de consumo indicados pela POF (2018)

In [30]:
df_feijao = df_feijao[df_feijao['Ano']>2017].reset_index(drop = True)
df_feijao

,Ano,Item,Produçao,Média móvel
0,2018,Feijão (em grão),2916365.0,2993444.6
1,2019,Feijão (em grão),2908075.0,2916142.4
2,2020,Feijão (em grão),3035290.0,2905415.2
3,2021,Feijão (em grão),3310916.2,3319211.0
4,2022,Feijão (em grão),3335580.2,3345889.0
5,2023,Feijão (em grão),3360244.2,3372567.0
6,2024,Feijão (em grão),3384908.2,3399245.0
7,2025,Feijão (em grão),3409572.2,3425923.0
8,2026,Feijão (em grão),3434236.2,3452601.0
9,2027,Feijão (em grão),3458900.2,3479279.0


Demanda populacional por FEIJÂO em toneladas (2018 -2030)

In [31]:
df_demanda_feijao = projecao_demanda_brasil[projecao_demanda_brasil['Items'] == 'Feijão'][['Ano', 'Items', 'Região', 'Demanda pop (ton)']].reset_index(drop = True)
df_demanda_feijao.head()

,Ano,Items,Região,Demanda pop (ton)
0,2018,Feijão,Brasil,1.039000e+06
1,2019,Feijão,Brasil,1.047233e+06
2,2020,Feijão,Brasil,1.055249e+06
3,2021,Feijão,Brasil,1.063033e+06
4,2022,Feijão,Brasil,1.070562e+06


Juntando os dataframe df_feijao e df_demanda_feijao

In [32]:
df_feijao = pd.merge(df_feijao, df_demanda_feijao, how = 'inner', on = 'Ano')
df_feijao

,Ano,Item,Produçao,Média móvel,Items,Região,Demanda pop (ton)
0,2018,Feijão (em grão),2916365.0,2993444.6,Feijão,Brasil,1.039000e+06
1,2019,Feijão (em grão),2908075.0,2916142.4,Feijão,Brasil,1.047233e+06
2,2020,Feijão (em grão),3035290.0,2905415.2,Feijão,Brasil,1.055249e+06
3,2021,Feijão (em grão),3310916.2,3319211.0,Feijão,Brasil,1.063033e+06
4,2022,Feijão (em grão),3335580.2,3345889.0,Feijão,Brasil,1.070562e+06
5,2023,Feijão (em grão),3360244.2,3372567.0,Feijão,Brasil,1.077817e+06
6,2024,Feijão (em grão),3384908.2,3399245.0,Feijão,Brasil,1.084794e+06
7,2025,Feijão (em grão),3409572.2,3425923.0,Feijão,Brasil,1.091495e+06
8,2026,Feijão (em grão),3434236.2,3452601.0,Feijão,Brasil,1.097911e+06
9,2027,Feijão (em grão),3458900.2,3479279.0,Feijão,Brasil,1.104034e+06


In [33]:
fig =px.line(df_feijao, x ='Ano', y = ['Produçao','Média móvel', 'Demanda pop (ton)'], title = 'Predição da produção e demanda de FEIJÃO - Brasil - (2018-2030)', labels= {'value': 'Produção (ton)'})
fig.update_layout(template = 'plotly_white')

* Cana

In [34]:
## Média móvel
df_cana['Média móvel'] = df_cana['Produçao'].rolling(window =5).mean()

In [35]:
fig =px.line(df_cana, x ='Ano', y=['Produçao', 'Média móvel'], title = 'Produção de CANA-DE-AÇUCAR - Brasil - (1974-2020)', labels= {'value': 'Produção (ton)'})
fig.update_layout(template = 'plotly_white')

In [36]:
#Cana
Y_cana = df_cana['Produçao'].values
X_cana = df_cana[['Ano']].values
X_cana = sm.add_constant(X_cana)

In [37]:
#Modelo
modelo_cana = sm.OLS(Y_cana,X_cana)
resultados_cana = modelo_cana.fit()
#print(resultados_cana.summary())

In [38]:
print("Coeficientes: ", resultados_cana.params)
print("R2: ", resultados_cana.rsquared)

Coeficientes:  [-3.18447702e+10  1.61453761e+07]
R2:  0.9141904702621043


* Café

In [39]:
## Média móvel
df_cafe['Média móvel'] = df_cafe['Produçao'].rolling(window =5).mean()

In [40]:
fig =px.line(df_cafe, x ='Ano', y=['Produçao', 'Média móvel'], title = 'Produção de CAFÉ - Brasil - (1974-2020)', labels= {'value': 'Produção (ton)'})
fig.update_layout(template = 'plotly_white')

In [41]:
#Cana
Y_cafe = df_cafe['Produçao'].values
X_cafe = df_cafe[['Ano']].values
X_cafe = sm.add_constant(X_cafe)

In [42]:
#Modelo
modelo_cafe = sm.OLS(Y_cafe,X_cafe)
resultados_cafe = modelo_cafe.fit()
#print(resultados_cana.summary())

In [43]:
print("Coeficientes: ", resultados_cafe.params)
print("R2: ", resultados_cafe.rsquared)

Coeficientes:  [-1.31400103e+07  7.97820398e+03]
R2:  0.02834240572380664


* Mandioca

In [44]:
## Média móvel
df_mandioca['Média móvel'] = df_mandioca['Produçao'].rolling(window =5).mean()

In [45]:
fig =px.line(df_mandioca, x ='Ano', y=['Produçao', 'Média móvel'], title = 'Produção de MANDIOCA - Brasil - (1974-2020)', labels= {'value': 'Produção (ton)'})
fig.update_layout(template = 'plotly_white')

In [46]:
#Mandioca
Y_mandioca = df_mandioca['Produçao'].values
X_mandioca = df_mandioca[['Ano']].values
X_mandioca = sm.add_constant(X_mandioca)

In [47]:
#Mandioca
modelo_mandioca = sm.OLS(Y_mandioca,X_mandioca)
resultados_mandioca = modelo_mandioca.fit()
#print(resultados_mandioca.summary())

In [48]:
print("Coeficientes: ", resultados_mandioca.params)
print("R2: ", resultados_mandioca.rsquared)

Coeficientes:  [ 1.70497266e+08 -7.38130933e+04]
R2:  0.1633307148277804


* Milho

In [49]:
## Média móvel
df_milho['Média móvel'] = df_milho['Produçao'].rolling(window =5).mean()

In [50]:
fig =px.line(df_milho, x ='Ano', y=['Produçao', 'Média móvel'], title = 'Produção de MILHO - Brasil - (1974-2020)', labels= {'value': 'Produção (ton)'})
fig.update_layout(template = 'plotly_white')

In [51]:
#Milho
Y_milho = df_milho['Produçao'].values
X_milho = df_milho[['Ano']].values
X_milho = sm.add_constant(X_milho)

In [52]:
#Milho
modelo_milho = sm.OLS(Y_milho,X_milho)
resultados_milho = modelo_milho.fit()
#print(resultados_milho.summary())

In [53]:
print("Coeficientes: ", resultados_milho.params)
print("R2: ", resultados_milho.rsquared)

Coeficientes:  [-3.25648651e+09  1.65134890e+06]
R2:  0.822357317432366


* Soja

In [54]:
## Média móvel
df_soja['Média móvel'] = df_soja['Produçao'].rolling(window =5).mean()

In [55]:
fig =px.line(df_soja, x ='Ano', y=['Produçao', 'Média móvel'], title = 'Produção de SOJA - Brasil - (1974-2020)', labels= {'value': 'Produção (ton)'})
fig.update_layout(template = 'plotly_white')

In [56]:
#Soja
Y_soja = df_soja['Produçao'].values
X_soja = df_soja[['Ano']].values
X_soja = sm.add_constant(X_soja)

In [57]:
#Soja
modelo_soja = sm.OLS(Y_soja, X_soja)
resultados_soja = modelo_soja.fit()
#print(resultados_soja.summary())

In [58]:
print("Coeficientes: ", resultados_soja.params)
print("R2: ", resultados_soja.rsquared)

Coeficientes:  [-4.48383732e+09  2.26657065e+06]
R2:  0.8525243653981363


* Tomate

In [59]:
## Média móvel
df_tomate['Média móvel'] = df_tomate['Produçao'].rolling(window =5).mean()

In [60]:
fig =px.line(df_tomate, x ='Ano', y=['Produçao', 'Média móvel'], title = 'Produção de TOMATE - Brasil - (1974-2020)', labels= {'value': 'Produção (ton)'})
fig.update_layout(template = 'plotly_white')

In [61]:
#Tomate
Y_tomate = df_tomate['Produçao'].values
X_tomate = df_tomate[['Ano']].values
X_tomate = sm.add_constant(X_tomate)

In [62]:
#Tomate
modelo_tomate = sm.OLS(Y_tomate, X_tomate)
resultados_tomate = modelo_tomate.fit()
#print(resultados_tomate.summary())

In [63]:
print("Coeficientes: ", resultados_tomate.params)
print("R2: ", resultados_tomate.rsquared)

Coeficientes:  [-1.47152442e+08  7.51013145e+04]
R2:  0.9450578331258392


* Trigo

In [64]:
## Média móvel
df_trigo['Média móvel'] = df_trigo['Produçao'].rolling(window =5).mean()

In [65]:
fig =px.line(df_trigo, x ='Ano', y=['Produçao', 'Média móvel'], title = 'Produção de TRIGO - Brasil - (1974-2020)', labels= {'value': 'Produção (ton)'})
fig.update_layout(template = 'plotly_white')

In [66]:
#Trigo
Y_trigo = df_trigo['Produçao'].values
X_trigo = df_trigo[['Ano']].values
X_trigo = sm.add_constant(X_trigo)

In [67]:
#Trigo
modelo_trigo = sm.OLS(Y_trigo, X_trigo)
resultados_trigo = modelo_trigo.fit()
#print(resultados_trigo.summary())

In [68]:
print("Coeficientes: ", resultados_trigo.params)
print("R2: ", resultados_trigo.rsquared)

Coeficientes:  [-1.44330753e+08  7.42328035e+04]
R2:  0.3790742501381902
